## STF Document Clustering using VICTOR Dataset
https://aclanthology.org/2020.lrec-1.181.pdf

Premissas do artigo
- To ensure the reproducibility of our experiments we ran
domly divided the appeals into 70%/15%/15% splits for
train/validation/test respectively, maintaining theme distri
bution across them.


In [ ]:
import pandas as pd
import plotly.express as px
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [1]:
df_doc = pd.read_csv('train_medium.csv')

In [37]:
df_doc.columns

Index(['themes', 'process_id', 'file_name', 'document_type', 'pages', 'body'], dtype='object')

In [38]:
df_doc.head()

,themes,process_id,file_name,document_type,pages,body
0,[225],AI_635072,AI_635072_15320124014_80_13032015.pdf,outros,1,"{""çft manê ado intimação extraído relação das ..."
1,[225],AI_635072,AI_635072_15320124014_80_13032015.pdf,outros,2,"{""certidão certifico que dirigi nesta capital ..."
2,[225],AI_635072,AI_635072_288462549_1060_13032015.pdf,outros,1,"{""supremo tribunal federal agravo instrumento ..."
3,[225],AI_635072,AI_635072_289561047_1280_15122014.pdf,outros,1,"{""supremo tribunal federal certidão agravo ins..."
4,[225],AI_635072,AI_635072_306972753_1280_02062015.pdf,outros,1,"{""supremo tribunal federal ofício brasília de ..."


In [39]:
df_view = df_doc.drop_duplicates(subset=["process_id", "file_name"])

In [40]:
doc_counts = df_view["document_type"].value_counts().reset_index()
doc_counts.columns = ["document_type", "count"]

fig = px.bar(
    doc_counts,
    x="document_type",
    y="count",
    title="Quantidade de documentos por tipo",
    text="count",
    color="document_type"
)

fig.update_layout(
    width=900,  
    height=600,  
)
fig.show()

Dados desbalanceados
- Muito dado pertence a classe outros
- Buscar formas de lidar com esses dados desbalanceados

In [41]:
df_view.head()

,themes,process_id,file_name,document_type,pages,body
0,[225],AI_635072,AI_635072_15320124014_80_13032015.pdf,outros,1,"{""çft manê ado intimação extraído relação das ..."
2,[225],AI_635072,AI_635072_288462549_1060_13032015.pdf,outros,1,"{""supremo tribunal federal agravo instrumento ..."
3,[225],AI_635072,AI_635072_289561047_1280_15122014.pdf,outros,1,"{""supremo tribunal federal certidão agravo ins..."
4,[225],AI_635072,AI_635072_306972753_1280_02062015.pdf,outros,1,"{""supremo tribunal federal ofício brasília de ..."
5,[225],AI_635072,AI_635072_306981455_1280_03062015.pdf,outros,1,"{""supremo tribunal federal ofício brasília de ..."


In [42]:
themes_counts = df_view["themes"].value_counts().reset_index()
themes_counts.columns = ["theme", "count"]

fig = px.bar(
    themes_counts.head(10),  
    x="theme",
    y="count",
    text="count",
    title="Temas mais frequentes na base VICTOR",
    color="theme"
)

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title="Tema", yaxis_title="Quantidade de documentos")

fig.show()

Tema 695: Inclusão do décimo terceiro salário no cálculo do salário-benefício para apuração da Renda Mensal Inicial (RMI) 

Tema 692: Possibilidade de o CONFEA fixar valores de taxas por resolução 

Tema 800: Presunção relativa de inexistência de repercussão geral de recursos extraordinários nos Juizados Especiais Cíveis (Lei 9.099/1995)

Tema 503: Apenas lei pode criar benefícios e vantagens previdenciárias no RGPS

Tema 163: Contribuição previdenciária sobre férias, gratificação natalina, horas extras, adicional noturno etc. 

Tema 540: Fixação de anuidade por conselhos de fiscalização profissional é inconstitucional quando delegada por lei

In [43]:
df_outros = df_view[df_view['document_type'] == 'outros']

themes_outros = df_outros["themes"].value_counts().reset_index()
themes_outros.columns = ["theme", "count"]


fig = px.bar(
    themes_outros.head(10),  
    x="theme",
    y="count",
    text="count",
    title="Temas mais frequentes na base VICTOR no tipo de documentos 'outros'",
    color="theme"
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title="Tema", 
    yaxis_title="Quantidade de documentos"
)

fig.show()

In [10]:
from nltk.tokenize import word_tokenize
import nltk
import string
import spacy
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unicodedata

### Tratamento de dados

In [2]:
# removendo a classe desbalanceada 
doc = df_doc[df_doc['document_type'] != "outros"]

In [3]:
doc.head()

,themes,process_id,file_name,document_type,pages,body
71,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,1,"{""poder judiciário estado acre gabinete desemb..."
72,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,2,"{""poder judiciário estado acre gabinete desemb..."
73,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,3,"{""poder judiciário estado acre gabinete desemb..."
74,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,4,"{""poder judiciário estado acre gabinete desemb..."
75,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,5,"{""c f d poder judiciário estado acre gabinete ..."


In [4]:
# "stopwords" brasileiras
with open("stopwords_br.txt", "r", encoding="utf-8") as f:
    stop_words = set([x.strip().lower() for x in f.readlines()])

In [5]:
# "stopwords" jurídicas
with open("stopwords_juridicas.txt", "r", encoding="utf-8") as f:
    juridicas = set([x.strip().lower() for x in f.readlines()])

In [6]:
# estados brasileiros e topicos comuns
with open("topicos_comuns.txt", "r", encoding="utf-8") as f:
    topicos_comuns = set([x.strip().lower() for x in f.readlines()])

In [7]:
stop_words = stop_words.union(juridicas)

In [8]:
def remove_accents(text: str) -> str:
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )


def normalize_spacing(text: str) -> str:
    invisible = [
        "\u00A0", "\u2000", "\u2001", "\u2002", "\u2003",
        "\u2004", "\u2005", "\u2006", "\u2007", "\u2008",
        "\u2009", "\u200A", "\u202F", "\u205F", "\u3000",
        "\u200B"     
    ]
    for sp in invisible:
        text = text.replace(sp, " ")

    text = re.sub(r"\s+", " ", text)
    return text.strip()

def normalize_citations(text: str) -> str:
    text = re.sub(r"\bart\.?\s*(\d+)\b", r"art_\1", text)
    text = re.sub(r"§\s*(\d+)", r"par_\1", text)

    roman_map = {
        "i":1, "ii":2, "iii":3, "iv":4, "v":5, "vi":6,
        "vii":7, "viii":8, "ix":9, "x":10, "xi":11, "xii":12
    }

    def roman_to_num(m):
        r = m.group(1).lower()
        return f"inc_{roman_map.get(r, r)}"

    text = re.sub(r"\binciso\s+([ivx]+)\b", roman_to_num, text)
    return text


def general_cleanup(text: str) -> str:

    patterns_remove = [
        r"http\S+",
        r"\S+@\S+",
        r"\(?\d{2}\)?\s?\d{4,5}-?\d{4}",
        r"\b\d{5}-?\d{3}\b",
        r"\b\d{3}\.?\d{3}\.?\d{3}-?\d{2}\b",
        r"\b\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}\b",
        r"\b\d{1,2}[\/\-.]\d{1,2}[\/\-.]\d{2,4}\b",
        r"\b\d{1,2}[:h]\d{2}(:\d{2})?\b",
    ]

    for p in patterns_remove:
        text = re.sub(p, " ", text)

    name_pattern = r"\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)+\b"
    text = re.sub(name_pattern, " ", text)

    return re.sub(r"\s+", " ", text).strip()


def normalize_token(w: str) -> str:
    return w.strip(".,;:!?()[]{}\"'`´“”‘’_-")


def remove_ocr_noise(text: str) -> str:
    clean = []

    for w in text.split():

        w_norm = normalize_token(w)

        if len(w_norm) <= 3:
            continue

        if len(w_norm) == 1:
            continue

        if len(set(w_norm)) <= len(w_norm) // 2:
            continue

        if not any(v in w_norm.lower() for v in "aeiou"):
            continue

        clean.append(w_norm)

    return " ".join(clean)



def preprocess(text: str) -> str:

    if not isinstance(text, str) or not text.strip():
        return ""

    text = normalize_spacing(text)      
    text = general_cleanup(text)
    text = remove_accents(text).lower()
    text = normalize_citations(text)
    text = normalize_spacing(text)     

    words = []
    for w in text.split():
        w_norm = normalize_token(w)

        if w_norm and w_norm not in stop_words and w_norm not in topicos_comuns:
            words.append(w_norm)

    text = " ".join(words)

    text = remove_ocr_noise(text)

    return text.strip()


In [17]:
doc["clean"] = doc["body"].apply(preprocess)

C:\Users\Maria\AppData\Local\Temp\ipykernel_28564\1964082047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc["clean"] = doc["body"].apply(preprocess)


In [18]:
doc.head()

,themes,process_id,file_name,document_type,pages,body,clean
71,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,1,"{""poder judiciário estado acre gabinete desemb...",declaracao apelacao civel orgao civel banco cr...
72,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,2,"{""poder judiciário estado acre gabinete desemb...",banco cruzeiro representante interpos presente...
73,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,3,"{""poder judiciário estado acre gabinete desemb...",obstante arrazoado recursal dessumo alegada co...
74,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,4,"{""poder judiciário estado acre gabinete desemb...",acolhimento aclaratorios edcl castro meira dec...
75,[33],AI_852811,AI_852811_941407_6_19052013.pdf,acordao_de_2_instancia,5,"{""c f d poder judiciário estado acre gabinete ...",extrato julgamento seguinte unanimidade conhec...


In [54]:
len(doc)

142435

In [55]:
doc['clean'][1466152]

'ante julgo improcedente extingo resolucao termos artigo_269 honorarios considerando percebe rendimento inferior salarios minimos criterio regiao defiro beneficio assistencia judiciaria gratuita havendo interposicao recebido efeitos oferecimento contrarrazoes decurso respectivo prazo remetam recursal subsecao judiciaria publicada registrada eletronicamente intimem transito julgado arquivem itajai eduardo correia silva substituto titularidade plena assinado eduardo correia silva substituto titularidade plena artigo_1o inc_3 lei_11419 resolucao regiao marco conferencia autenticidade disponivel endereco site preenchimento codigo verificador solicitado codigo adicionais eduardo correia silva'

### Bertopic 

In [56]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer


In [57]:
stop_words_list = list(stop_words)

In [99]:
treino = doc[["clean", "document_type"]]

In [100]:
treino.head()

,clean,document_type
71,declaracao apelacao civel orgao civel banco cr...,acordao_de_2_instancia
72,banco cruzeiro representante interpos presente...,acordao_de_2_instancia
73,obstante arrazoado recursal dessumo alegada co...,acordao_de_2_instancia
74,acolhimento aclaratorios edcl castro meira dec...,acordao_de_2_instancia
75,extrato julgamento seguinte unanimidade conhec...,acordao_de_2_instancia


In [101]:
treino['document_type'].value_counts()

document_type
peticao_do_RE                       77893
agravo_em_recurso_extraordinario    34640
sentenca                            21210
acordao_de_2_instancia               4740
despacho_de_admissibilidade          3952
Name: count, dtype: int64

In [102]:
docs = list(treino["clean"])  

In [103]:
len(docs)

142435

In [104]:
# amostra estratificada
sample_df, _ = train_test_split(
    treino,
    test_size=0.8, 
    stratify=treino["document_type"],
    random_state=42
)
documents = list(sample_df["clean"])

In [105]:
documents

['protege particular interesses corretor ajuste verbal restar descumprido dispositivos legais autorizarao corretor promova demanda resguardo direitos unico civil discipline taxativamente comissao devera necessariamente suportada contrato mediacao tornou possivel autorize considerar nula convencao sentido contrario vale citar sentido escolio orlando gomes determinando pagar corretagem prevalecem usos livres todavia contrato mediacao estipulares clausula remuneracao ajuste recorre usos recorrido proferida juizo acabou silenciar venias rogadas suscitadas tendo fundado encontro contidas impor graves prejuizos recorrentes condenadas restituir valores comprovadamente receberam colocaria risco desenvolvimento atividade certo constatar desconhecimento consumidor contratacao profissional corretagem ilegalidade pagamento invalidando negocio juridico firmado recorrida corretora imobiliaria especifica impugnacao trazidos ausente casu condenacao imposta recorrentes inadequada porquanto infringir co

In [123]:
topic_model = BERTopic(min_topic_size=150, embedding_model="paraphrase-multilingual-MiniLM-L12-v2", 
                       language="portuguese")

In [124]:
topics, probs = topic_model.fit_transform(documents)

In [125]:
topic_model.visualize_topics()

In [127]:
topic_model.get_topic_freq()

,Topic,Count
0,0,21000
2,-1,4378
1,1,2046
3,2,519
4,3,236
6,4,156
5,5,152


In [128]:
topic_model.visualize_barchart()

In [139]:
topic_model.save("bertopic_model")

2025-11-22 23:44:57,253 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [150]:
from bertopic import BERTopic

model = BERTopic.load("bertopic_model", embedding_model="paraphrase-multilingual-MiniLM-L12-v2")

In [154]:
# descobrindo os tópicos presentes na classe "outros"
outros = df_doc[df_doc['document_type'] == "outros"]

In [155]:
outros.head()

,themes,process_id,file_name,document_type,pages,body
0,[225],AI_635072,AI_635072_15320124014_80_13032015.pdf,outros,1,"{""çft manê ado intimação extraído relação das ..."
1,[225],AI_635072,AI_635072_15320124014_80_13032015.pdf,outros,2,"{""certidão certifico que dirigi nesta capital ..."
2,[225],AI_635072,AI_635072_288462549_1060_13032015.pdf,outros,1,"{""supremo tribunal federal agravo instrumento ..."
3,[225],AI_635072,AI_635072_289561047_1280_15122014.pdf,outros,1,"{""supremo tribunal federal certidão agravo ins..."
4,[225],AI_635072,AI_635072_306972753_1280_02062015.pdf,outros,1,"{""supremo tribunal federal ofício brasília de ..."


In [156]:
len(outros)

1323841

In [157]:
# amostra aleatória de documentos "outros"
outros = outros.sample(n=100000, random_state=42)

In [158]:
outros["clean"] = outros["body"].apply(preprocess)

In [159]:
outros = outros[["clean", "document_type"]]

In [160]:
doc_outros = list(outros["clean"])

In [161]:
topics_outros, probs_outros = model.fit_transform(doc_outros)

In [162]:
model.get_topic_freq()

,Topic,Count
4,-1,48091
5,0,3391
17,1,2975
10,2,2565
14,3,2405
...,...,...
65,84,161
23,85,159
25,86,158
89,87,158


In [163]:
model.visualize_barchart()

### LLM + FIRAC com Few Shot Learning

In [164]:
! pip install langchain openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ---------------------------------------- 42.9/42.9 kB 1.1 MB/s eta 0:00:00
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/93.7 kB ? eta -:--:--
   ---------------------------------------- 93.7/93.7 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   --------------------------------- ------ 0.9/1.0 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 12.8 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   -------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
system_prompt = """Você é um assistente jurídico que gera FIRAC (Facts, Issues, Rules, Analysis, Conclusion)
a partir de textos jurídicos limpos de OCR. Retorne um JSON válido com os campos:
- facts
- issues
- rules
- analysis
- conclusion
Mantenha a linguagem concisa e objetiva. Trate o texto como um único bloco. Não invente informações. 
Caso não haja informação suficiente, retorne campos vazios."""


# few-shot learning
examples = """Exemplo:
Texto: autor ajuizou acao indenizacao contrato descumprido clausula rescisao
Saída esperada:
{
  "facts": [
    {"text":"Autor ajuizou ação de indenização"},
    {"text":"Cláusula de rescisão prevista"}
  ],
  "issues":[{"text":"Se a rescisão contratual foi válida"}],
  "rules":[{"text":"Art. 421 CC","authority":"Código Civil"}],
  "analysis":[{"point":"Aplicando os fatos à lei, a cláusula é válida conforme jurisprudência"}],
  "conclusion":"Pedido de indenização deferido"
}"""

In [19]:
texto = doc['clean'].iloc[100]

In [20]:
texto

'incidir diploma civil conseguinte afasto prejudicial plano verao expurgos inflacionarios plano verao importante diga referencia cita termo inicial apresentou extrato contas constava periodo modo calculos elaborados contador baseados saldos encontrados periodos plano collor collor tendo concordado respectivos valores plano collor plano economico governamental collor anunciado fernando collor mello tentativa estancar fenomeno historico inflacao assolava pais plano previu dentre medidas bloqueio saldos cadernetas poupanca contas correntes aplicacoes overnight plano acarretou expurgos inflacionarios bancos detrimento poupadores tais expurgos base adquirido poupadores ocorreu periodos plano bresser plano verao lacuna legislacao previa alteracao indices promulgacao medida provisoria cadernetas poupanca remuneradas base regra artigo_17 lei_7730 artigo_17 saldos cadernetas poupanca serao atualizados base variacao verificada anterior editada medida provisoria posteriormente convertida lei_8024

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_KEY")

In [21]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"{examples}\n\nTexto novo:\n{texto}"}
]

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

print(response.choices[0].message.content)

{
  "facts": [
    {"text": "Plano Verão e Plano Collor implementados para conter inflação."},
    {"text": "Medidas incluíram bloqueio e expurgos inflacionários em cadernetas de poupança e contas correntes."},
    {"text": "Saldos das cadernetas de poupança deveriam ser atualizados com base na variação fiscal e convertidos segundo limites legais."},
    {"text": "Lei 7.730 e Medida Provisória posteriormente convertida na Lei 8.024 regulavam atualização e conversão dos saldos."},
    {"text": "Quantias excedentes ao limite de 50 cruzados deveriam ser recolhidas ao Banco Central e liberadas em parcelas mensais."},
    {"text": "Banco Central estabeleceu regras de atualização monetária e juros equivalentes aos saldos convertidos."},
    {"text": "Alterações legislativas posteriores modificaram redação do artigo 6º, porém conversões perderam eficácia por revogação e decisões do Congresso."},
    {"text": "Lacuna legislativa persistiu quanto à correção monetária dos valores mantidos em con